<a href="https://colab.research.google.com/github/DeoZD/CSMODEL_G2_MCO/blob/main/CSMODEL_G2_MCO1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSMODEL MCO 1 Group 2**
## LaSalleGameKNB?


In [ ]:
import pandas as pd